Работа с векторными базами данных

1. Загрузка датасета

In [6]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer #модель эмбеддингов
from langchain_core.vectorstores import InMemoryVectorStore # библиотека векторог поиска
from langchain_core.documents import Document 


In [7]:
dataset = load_dataset("ai-forever/ria-news-retrieval")
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 10000
    })
})


2. Выбор модели эмбенддингов

Модель преобразует текст в числовые векторы (эмбеддинги), чтобы можно было сравнивать вопросы с текстами корпуса по смыслу.

Результат:
Готовая модель для создания векторных представлений текстов.

In [8]:

embedding_model = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v1")


3. Создание векторной базы

In [11]:

# Создание векторного хранилища InMemoryVectorStore с указанием модели эмбеддингов
vector_store = InMemoryVectorStore(embedding_model)

# Преобразуем корпус в объекты Document с метаданными (id) и добавляем в векторную базу
corpus_texts = [x["text"] for x in dataset["corpus"]]
docs = [Document(page_content=text, metadata={"id": i}) for i, text in enumerate(corpus_texts)]
print(f"Добавляем {len(docs)} документов в векторное хранилище...")
vector_store.add_documents(docs)

KeyError: 'corpus'

4. Проверка поиска

In [ ]:
query_text = dataset["queries"][0]["text"]
results = vector_store.similarity_search(query_text, k=3)  # top-3 похожих
print(results) #список из 3-х текстов, наиболее похожих на запрос
#оценка качесва с default
correct_id = dataset["default"][0]["corpus_id"]
top_result_ids = [r.metadata['id'] for r in results]

# Проверяем, есть ли правильный ответ в top-1, top-3
top1_correct = correct_id == top_result_ids[0]
top3_correct = correct_id in top_result_ids
print(f"Top-1 correct: {top1_correct}, Top-3 correct: {top3_correct}")


In [ ]:
# Подготовка переменных для подсчета ошибок
correct_top1 = 0
correct_top3 = 0
total_queries = len(dataset["queries"])
errors = []  # сюда будем добавлять неудачные запросы

# Прогон по всем вопросам
for i, query in enumerate(dataset["queries"]):
    query_text = query["text"]
    correct_id = dataset["default"][i]["corpus_id"]

    # Поиск top-3
    results = vector_store.similarity_search(query_text, k=3)

    # Логирование top-3 результатов с текстом
    top3_texts = [r.page_content[:100] + ("..." if len(r.page_content) > 100 else "") for r in results]
    top_result_ids = [r.metadata['id'] for r in results]

    # Проверка
    top1_correct = correct_id == top_result_ids[0]
    top3_correct = correct_id in top_result_ids

    if top1_correct:
        correct_top1 += 1
    if top3_correct:
        correct_top3 += 1
    else:
        errors.append({
            "query_index": i,
            "query_text": query_text,
            "correct_id": correct_id,
            "top_results": top3_texts
        })

    # Красивое логирование
    print(f"--- Query {i} ---")
    print(f"Text: {query_text[:150]}{'...' if len(query_text) > 150 else ''}")
    print(f"Correct corpus ID: {correct_id}")
    print("Top-3 results:")
    for rank, (doc_id, snippet) in enumerate(zip(top_result_ids, top3_texts), 1):
        print(f"  {rank}. ID {doc_id}: {snippet}")
    print(f"Top-1 correct: {top1_correct}, Top-3 correct: {top3_correct}")
    print("-" * 80)

# Итоговая статистика
print("\n=== Итоговая статистика ===")
print(f"Всего запросов: {total_queries}")
print(f"Top-1 accuracy: {correct_top1}/{total_queries} = {correct_top1 / total_queries:.2%}")
print(f"Top-3 accuracy: {correct_top3}/{total_queries} = {correct_top3 / total_queries:.2%}")
print(f"Ошибки: {len(errors)}")

# Пример ошибок (первые 3)
if errors:
    print("\nПримеры ошибок:")
    for e in errors[:3]:
        print(f"\nQuery {e['query_index']}: {e['query_text'][:150]}...")
        print(f"Correct ID: {e['correct_id']}")
        print("Top-3 results (snippets):")
        for snippet in e['top_results']:
            print(f"  - {snippet}")